<a href="https://colab.research.google.com/github/NelsonN2015/IBM-Jupyter-Notebook-Lab/blob/main/clean_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import re
import os
import fitz  # PyMuPDF
import csv
import json
import pandas as pd

# Definindo as expressões regulares para os atributos
foro_regex = re.compile(r"FORO\s*(?:DA|DE)\s*(.*?)(\n|$)", re.IGNORECASE)
juiz_regex = re.compile(r"Juiz(?:\(a\))?\s*de\s*Direito[:\s]*Dr(?:\(a\))?[.\s]*([A-ZÁÉÍÓÚÂÊÎÔÛÃÕÇÑa-záéíóúâêîôûãõçñ\s]+)(\n|$)", re.IGNORECASE)
corpo_sentenca_regex = re.compile(r"Vistos\s*(.*?)\s*(?:DOCUMENTO ASSINADO DIGITALMENTE|Assinatura Digital|Este documento é cópia do original)", re.DOTALL | re.IGNORECASE)
resultado_regex = re.compile(r"\b(PROCEDENTE|IMPROCEDENTE|PARCIALMENTE\s*PROCEDENTE)\b", re.IGNORECASE)

# Função para limpar o texto
def limpar_texto(texto):
    # Remover URLs
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)
    # Remover avisos de assinatura digital
    texto = re.sub(r'Este documento é cópia do original, assinado digitalmente', '', texto, flags=re.IGNORECASE)
    # Remover qualquer outra linha padrão de rodapé
    texto = re.sub(r'Para conferir o original.*', '', texto, flags=re.IGNORECASE)
    return texto

# Função para normalizar e limpar o texto removendo caracteres especiais
def normalizar_texto(texto):
    texto = texto.lower()  # Converter para minúsculas
    texto = texto.strip()  # Remover espaços extras
    return texto

# Função para extrair os dados de uma sentença judicial
def extrair_dados(sentenca):
    sentenca = limpar_texto(sentenca)

    foro = foro_regex.search(sentenca)
    juiz = juiz_regex.search(sentenca)
    corpo_sentenca = corpo_sentenca_regex.search(sentenca)
    resultado = resultado_regex.search(sentenca)

    dados = {
        "Foro": normalizar_texto(foro.group(1).strip()) if foro else None,
        "Juiz de Direito": normalizar_texto(juiz.group(1).strip()) if juiz else None,
        "Corpo da Sentença": normalizar_texto(corpo_sentenca.group(1).strip()) if corpo_sentenca else None,
        "Resultado": normalizar_texto(resultado.group(1).strip()) if resultado else None,
    }
    return dados

# Caminho para o diretório contendo os arquivos de sentenças
diretorio_raw = '/content/raw'

# Lista para armazenar os resultados
resultados = []

# Função para ler o texto de um arquivo PDF
def ler_texto_pdf(caminho_arquivo):
    texto = ""
    try:
        doc = fitz.open(caminho_arquivo)
        for pagina in doc:
            texto += pagina.get_text()
    except Exception as e:
        print(f"Erro ao ler o arquivo {caminho_arquivo}: {e}")
    return texto

# Processando todos os arquivos no diretório raw
for nome_arquivo in os.listdir(diretorio_raw):
    caminho_arquivo = os.path.join(diretorio_raw, nome_arquivo)
    if os.path.isfile(caminho_arquivo):
        if caminho_arquivo.lower().endswith('.pdf'):
            conteudo = ler_texto_pdf(caminho_arquivo)
            if conteudo:
                dados_extracao = extrair_dados(conteudo)
                if any(dados_extracao.values()):  # Verifica se pelo menos um valor foi extraído
                    resultados.append(dados_extracao)
                else:
                    print(f"Nenhum dado extraído do arquivo {nome_arquivo}")
            else:
                print(f"Conteúdo vazio no arquivo {nome_arquivo}")
        else:
            print(f"Arquivo ignorado (não é PDF): {nome_arquivo}")

# Converter a lista de resultados para um DataFrame
df_resultados = pd.DataFrame(resultados)

# Remover sentenças com valores nulos em qualquer coluna
df_resultados.dropna(inplace=True)

# Verificar se há resultados após a remoção
if not df_resultados.empty:
    # Exibir os resultados
    print(df_resultados)

    # Salvando resultados em CSV
    try:
        df_resultados.to_csv('resultados.csv', index=False, encoding='utf-8')
        print("Arquivo CSV criado com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar arquivo CSV: {e}")

    # Salvando resultados em JSON
    try:
        df_resultados.to_json('resultados.json', orient='records', force_ascii=False, indent=4)
        print("Arquivo JSON criado com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar arquivo JSON: {e}")
else:
    print("Nenhum dado foi extraído.")


Nenhum dado extraído do arquivo doc_56250237.pdf
Nenhum dado extraído do arquivo doc_56290024.pdf
Nenhum dado extraído do arquivo doc_46083588.pdf
Nenhum dado extraído do arquivo doc_56228762.pdf
Nenhum dado extraído do arquivo doc_46090117.pdf
Nenhum dado extraído do arquivo doc_56292679.pdf
Nenhum dado extraído do arquivo doc_56286539.pdf
Nenhum dado extraído do arquivo doc_46087731.pdf
Nenhum dado extraído do arquivo doc_46085104.pdf
                       Foro                        Juiz de Direito  \
2                 andradina         paulo victor alvares gonçalves   
10                  cajamar               marcelo henrique mariano   
12                 sorocaba                  gustavo scaf de molon   
29           novo horizonte  gislaine de brito faleiros vendramini   
30    são josé do rio preto       andressa maria tavares marchiori   
...                     ...                                    ...   
3060        itaquaquecetuba                  kleber leles de souza   

In [20]:
import pandas as pd

# Caminho para o arquivo JSON
caminho_arquivo_json = 'resultados.json'

# Ler o arquivo JSON
df = pd.read_json(caminho_arquivo_json)

# Exibir as primeiras linhas do DataFrame
print(df.head())


                    Foro                        Juiz de Direito  \
0              andradina         paulo victor alvares gonçalves   
1                cajamar               marcelo henrique mariano   
2               sorocaba                  gustavo scaf de molon   
3         novo horizonte  gislaine de brito faleiros vendramini   
4  são josé do rio preto       andressa maria tavares marchiori   

                                   Corpo da Sentença                Resultado  
0  .\ni  relatório\nmario correia de andrade prop...             improcedente  
1  .\ntrata-se de pedido de exibição dos contrato...             improcedente  
2  .\ninicialmente, necessário a retificação do p...             improcedente  
3  . \nluzia macedo da silva ajuizou ação de inde...             improcedente  
4  .\ntrata-se de ação declaratória de inexistênc...  parcialmente procedente  


In [21]:
import json

# Caminho para o arquivo JSON
caminho_arquivo_json = 'resultados.json'  # Ajuste o caminho conforme necessário

# Ler o arquivo JSON
with open(caminho_arquivo_json, 'r', encoding='utf-8') as file:
    dados = json.load(file)

# Exibir os primeiros registros
print("Usando json:")
print(dados[:5])  # Exibe os primeiros 5 registros


Usando json:
[{'Foro': 'andradina', 'Juiz de Direito': 'paulo victor alvares gonçalves', 'Corpo da Sentença': '.\ni  relatório\nmario correia de andrade propôs ação declaratória de inexistência de \ndébito c/c indenização por danos materiais e morais em face de banco agibank s/a.\nalega que foi realizado um empréstimo consignado em seu nome pelo banco \nrequerido, cujo contrato de nº 1218365725 não reconhece. pede a declaração de inexigibilidade do \ncontrato. requer a inversão do ônus da prova. pugna pela condenação da ré à devolução em dobro \ndos valores cobrados, além de indenização por danos morais e, por fim, a condenação da parte \nrequerida ao pagamento das custas processuais e honorários advocatícios. juntou documentos (fls. \n28/106).\na inicial foi recebida, tendo sido concedida assistência judiciária gratuita à parte \nautora (fls. 107).\ncitado (fls. 111), o banco requerido contestou (fls. 209/217). alega que o \nempréstimo consignado foi firmado por meio de contrato digit

In [22]:
import pandas as pd

# Caminho para o arquivo JSON
caminho_arquivo_json = 'resultados.json'  # Ajuste o caminho conforme necessário

# Ler o arquivo JSON e criar um DataFrame
df = pd.read_json(caminho_arquivo_json)

# Exibir as primeiras linhas do DataFrame
print(df.head())


                    Foro                        Juiz de Direito  \
0              andradina         paulo victor alvares gonçalves   
1                cajamar               marcelo henrique mariano   
2               sorocaba                  gustavo scaf de molon   
3         novo horizonte  gislaine de brito faleiros vendramini   
4  são josé do rio preto       andressa maria tavares marchiori   

                                   Corpo da Sentença                Resultado  
0  .\ni  relatório\nmario correia de andrade prop...             improcedente  
1  .\ntrata-se de pedido de exibição dos contrato...             improcedente  
2  .\ninicialmente, necessário a retificação do p...             improcedente  
3  . \nluzia macedo da silva ajuizou ação de inde...             improcedente  
4  .\ntrata-se de ação declaratória de inexistênc...  parcialmente procedente  


In [23]:
df.head(n=10)

,Foro,Juiz de Direito,Corpo da Sentença,Resultado
0,andradina,paulo victor alvares gonçalves,.\ni relatório\nmario correia de andrade prop...,improcedente
1,cajamar,marcelo henrique mariano,.\ntrata-se de pedido de exibição dos contrato...,improcedente
2,sorocaba,gustavo scaf de molon,".\ninicialmente, necessário a retificação do p...",improcedente
3,novo horizonte,gislaine de brito faleiros vendramini,. \nluzia macedo da silva ajuizou ação de inde...,improcedente
4,são josé do rio preto,andressa maria tavares marchiori,.\ntrata-se de ação declaratória de inexistênc...,parcialmente procedente
5,pereira barreto,mateus moreira siketo,". \ndispensado o relatório, nos termos do arti...",improcedente
6,osasco,denise indig pinheiro,.\ntrata-se de ação de obrigação de fazer e de...,procedente
7,mirassol,natália berti,".\nrelatório dispensado, nos termos do art. 38...",parcialmente procedente
8,osasco,denise indig pinheiro,.\ntrata-se de ação ordinária ajuizada por d. ...,procedente
9,mirassol,andre da fonseca tavares,.\ntratar-se de ação de indenização por danos ...,parcialmente procedente


In [24]:
df.shape # retorna uma tupla (qtd linhas, qtd colunas)

(502, 4)

In [26]:
df.head(n=5)

,Foro,Juiz de Direito,Corpo da Sentença,Resultado
0,andradina,paulo victor alvares gonçalves,.\ni relatório\nmario correia de andrade prop...,improcedente
1,cajamar,marcelo henrique mariano,.\ntrata-se de pedido de exibição dos contrato...,improcedente
2,sorocaba,gustavo scaf de molon,".\ninicialmente, necessário a retificação do p...",improcedente
3,novo horizonte,gislaine de brito faleiros vendramini,. \nluzia macedo da silva ajuizou ação de inde...,improcedente
4,são josé do rio preto,andressa maria tavares marchiori,.\ntrata-se de ação declaratória de inexistênc...,parcialmente procedente


In [27]:
df.dtypes

Foro                 object
Juiz de Direito      object
Corpo da Sentença    object
Resultado            object
dtype: object

In [28]:
df['Foro'] = df['Foro'].astype('category')
df['Juiz de Direito'] = df['Juiz de Direito'].astype('category')
df['Resultado'] = df['Resultado'].astype('category')


In [29]:
df.dtypes

Foro                 category
Juiz de Direito      category
Corpo da Sentença      object
Resultado            category
dtype: object

In [30]:
# Exibir informações do DataFrame
print(df.info())

# Exibir as primeiras linhas do DataFrame
print(df.head())

# Operações de análise e manipulação
# Por exemplo, contar a frequência dos resultados
resultado_contagem = df['Resultado'].value_counts()
print(resultado_contagem)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Foro               502 non-null    category
 1   Juiz de Direito    502 non-null    category
 2   Corpo da Sentença  502 non-null    object  
 3   Resultado          502 non-null    category
dtypes: category(3), object(1)
memory usage: 17.4+ KB
None
                    Foro                        Juiz de Direito  \
0              andradina         paulo victor alvares gonçalves   
1                cajamar               marcelo henrique mariano   
2               sorocaba                  gustavo scaf de molon   
3         novo horizonte  gislaine de brito faleiros vendramini   
4  são josé do rio preto       andressa maria tavares marchiori   

                                   Corpo da Sentença                Resultado  
0  .\ni  relatório\nmario correia de andrade prop...        

In [33]:
import os

pasta = '/content/raw'

# Lista todos os arquivos na pasta
arquivos = os.listdir(pasta)

# Conta apenas os arquivos (ignora subpastas)
numero_arquivos = len([arquivo for arquivo in arquivos if os.path.isfile(os.path.join(pasta, arquivo))])

print(f"Existem {numero_arquivos} arquivos na pasta '{pasta}'")

Existem 3602 arquivos na pasta '/content/raw'
